Code portions:
- All Pale chapters (through 23.1)
- Reasonably "optimal" preprocessing parameters
- fine-tuned embedding model
- ElasticSearch container Document Store
- Iterate and add other series embeddings?

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%bash

nvidia-smi


Sat Mar 11 03:35:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    24W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install "faiss-gpu>=1.6.3,<2"
!pip install farm-haystack[colab,pinecone]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import pickle
import logging
import time
from haystack.document_stores import PineconeDocumentStore
from haystack import Document
from haystack.nodes import PreProcessor, EmbeddingRetriever, Seq2SeqGenerator, TransformersSummarizer, FARMReader
from haystack.pipelines import GenerativeQAPipeline, ExtractiveQAPipeline, SearchSummarizationPipeline


In [5]:
os.chdir('./drive/MyDrive/pale-companion-files/full/')

In [6]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.WARNING)


In [7]:
# print(os.getcwd())
# os.chdir('./drive/MyDrive/pale-companion-files/es-experimentation/')

# print(os.getcwd())

In [8]:
MODEL_LOCATION = "../finetune/twig_otherverse_parahumans_adapted/"
PALE_CHAPTERS_FNAME = "../chapter_fmt_list.pkl"
# TODO add serial name to the metadata

In [9]:
with open(PALE_CHAPTERS_FNAME,'rb') as f:
    all_chapters = pickle.load(f)
# chapters = [i for i in all_chapters if int(i['meta']['arc_number']) < 3]
chapters = all_chapters
print(f"Testing with {len(chapters)} chapters")

Testing with 307 chapters


In [10]:
chapters[0]['meta']

{'arc_title': 'Blood Runs Cold',
 'pov': 'Louise',
 'wordcount': '7174',
 'series_chapter_number': 1,
 'arc_number': '0',
 'extra_material': False,
 'title': 'Blood Runs Cold - 0.0',
 'chapter': '0.0'}

In [11]:
preprocessor = PreProcessor(
    split_by='word',
    split_length=250,
    split_overlap=20,

    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_respect_sentence_boundary=True,
    progress_bar=True, 
    add_page_number=True
)
docs = preprocessor.process(chapters)
print(f"We will be working with {len(docs)} documents from {len(all_chapters)} chapters")

Preprocessing:   0%|          | 0/307 [00:00<?, ?docs/s]

We will be working with 15203 documents from 307 chapters


In [13]:
document_store = PineconeDocumentStore(
    api_key='6dcb03dd-509b-4ad3-8342-b705027cb07c',
    similarity='cosine',
    duplicate_documents='skip',
    embedding_field="emb",
    embedding_dim=768,
    metadata_config = {'indexed':['series_chapter_number','chapters','title']},
    index='all_pale_docs'
)

document_store.write_documents(docs, batch_size=32)

Writing Documents:   0%|          | 0/15203 [00:00<?, ?it/s]

In [16]:
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=MODEL_LOCATION,
    model_format="sentence_transformers",
    max_seq_len=500,
    progress_bar=True,
)


document_store.update_embeddings(retriever,update_existing_embeddings=False, batch_size=32)

Updating Embedding:   0%|          | 0/15203 [00:00<?, ? docs/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa", num_beams=8, max_length=500)
# Eventually with want to tweak the above
pipe = GenerativeQAPipeline(generator, retriever) # We specify the params later

In [18]:
TEST_QUESTION = "Who is Avery Kelly?"

TEST_OUTPUT = pipe.run(
                query = TEST_QUESTION,
                params = {
                    "Retriever": {"top_k":10},
                    # "Generator": {"max_length": maxlength, "num_beams": nbeam}
                }
            )

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
TEST_OUTPUT

{'query': 'Who is Avery Kelly?',
 'answers': [<Answer {'answer': 'Avery Kelly is one of the most famous people in the history of the United States. She was the first woman to be awarded the Medal of Honor for her service in the Korean War. She is also the first person to have been awarded the Order of the Purple Heart, which is the highest civilian honor in the US.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['bf94ec4d17ea39e9a5ded20ee2be8b04', 'e86a063665af1e22e1e8ea92c4f5d6d4', '990ab27cc86c43a88783c915e2467769', 'a3fd13f37e86101c5f0fb9614c9862e5', '3b90016a147875d0ef6bb60080e40bac', 'e7b3ba8b4dab2d1a30ce13d501199d21', '3371f1791e6b73d7b16e1c20d0b6c6e9', '14938c80d6674191d571319ea5ffe775', '2ee57ccbd965e6c0a0bc78d9222af926', 'a7ad8c7e4f133f7abd4924295c4e3ffd'], 'meta': {'doc_scores': [0.7749020755, 0.7630667090000001, 0.756303221, 0.7553996445, 0.746283293, 0.745530024, 0.744013354, 0.7372856585, 0.7